## TensorFlow基础

TensorFlow提供了强大的自动求导机制来计算导数。以下代码展示了如何使用tf.GradientTape()计算函数$y(x)=x^2$在$x=3$时的导数：

In [29]:
import tensorflow as tf
tf.enable_eager_execution()

In [30]:
x = tf.get_variable('x', shape=[1], initializer=tf.constant_initializer(3.))

In [31]:
# 在tf.GradientTape()的上下文内，所有计算步骤都会被记录以用于求导
with tf.GradientTape() as tape:
    y = tf.square(x)

In [32]:
y_grad = tape.gradient(y, x)

In [33]:
y

<tf.Tensor: id=93, shape=(1,), dtype=float32, numpy=array([9.], dtype=float32)>

In [34]:
y.numpy()

array([9.], dtype=float32)

In [35]:
y_grad.numpy()

array([6.], dtype=float32)

在机器学习中，更加常见的时是对多元函数求偏导，以及对向量或矩阵的求导。以下代码展示了如何使用`tf.GradientTape()`计算函数$L(w,b)=||Xw+b-y||^2$在$w=(1,2)^T,b=1$时分别对$w,b$的偏导数。其中$X=\begin{bmatrix}1&2\\3&4\end{bmatrix},y=\begin{bmatrix}1\\2\end{bmatrix}$。

In [36]:
X = tf.constant([[1., 2.], [3., 4.]])

In [37]:
y = tf.constant([[1.], [2.]])

In [38]:
w = tf.get_variable('w', shape=(2,1), initializer=tf.constant_initializer([[1.], [2.]]))

In [44]:
b = tf.get_variable('b', shape=(), initializer=tf.constant_initializer([1.]))

In [46]:
with tf.GradientTape() as tape:
    L = 0.5 * tf.reduce_sum(tf.square(tf.matmul(X, w) + b -y))

In [47]:
# 计算L(w, b)关于w，b的偏导数
w_grad, b_grad = tape.gradient(L, (w, b))

In [49]:
L.numpy()

62.5

In [50]:
w_grad.numpy()

array([[35.],
       [50.]], dtype=float32)

In [51]:
b_grad.numpy()

15.0

### 基础示例：线性回归

考虑一个实际问题，某城市在2013年-2017年的房价如下表所示：


|年份|2013|2014|2015|2016|2017|
|:-:|:-:|:-:|:-:|:-:|:-:|
|房价|12000|14000|15000|16500|17500|
现在，我们希望通过对该数据进行线性回归，即使用模型$y=ax+b$来拟合上述数据，此处$a$和$b$是待求的参数。

首先，我们定义数据，进行基本的归一化操作。

In [52]:
import numpy as np

In [56]:
x_raw = np.array([2013, 2014, 2015, 2016, 2017], dtype=float)

In [57]:
y_raw = np.array([12000, 14000, 15000, 16500, 17500], dtype=float)

In [59]:
x = (x_raw - x_raw.min()) / (x_raw.max() - x_raw.min())

In [60]:
y = (y_raw - y_raw.min()) / (y_raw.max() - y_raw.min())

在以下代码中，我们手工求损失函数关于参数$a$和$b$的偏导数，并用梯度下降法反复迭代，最终获得$a$和$b$的值。

In [61]:
a, b = 0, 0

In [62]:
num_epoch = 10000

In [63]:
learning_rate = 1e-3  # 学习率为1 * 10^(-3)

In [64]:
for e in range(num_epoch):
    # 手动计算损失函数关于自变量（模型参数）的梯度
    y_pre = a * x + b
    grad_a, grad_b = (y_pre -y).dot(x), (y_pre -y).sum()
    # 更新参数
    a, b = a - learning_rate * grad_a, b - learning_rate * grad_b

In [66]:
a, b

(0.9763702087567374, 0.05756498076575413)

TensorFlow的Eager Execution（动态图）模式与上述NumPy的运行方式类似，然而提供了更快速的运算（GPU支持）、自动求导、优化器等一系列对深度学习非常重要的功能。这里，TensorFlow帮助我们做了两件重要的工作：
* 使用`tape.gradient(ys, xs)`自动计算梯度；
* 使用`optimizer.apply_gradients(grads_and_vars)`自动更新模型参数。

In [70]:
x = tf.constant(x)
y = tf.constant(y)

a = tf.get_variable('a', dtype=tf.float64, shape=(), initializer=tf.zeros_initializer)  # 这里因为x_raw设置的float类型为64bits
b = tf.get_variable('b', dtype=tf.float64, shape=(), initializer=tf.zeros_initializer)
variables = (a, b)

num_epoch = 10000
# 声明一个梯度下降优化器
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-3)

In [71]:
for e in range(num_epoch):
    # 使用tf.GradientTape()记录损失函数的梯度信息
    with tf.GradientTape() as tape:
        y_pre = a * x + b
        loss = 0.5 * tf.reduce_sum(tf.square(y_pre - y))
    # TensorFlow自动计算损失函数关于自变量（模型参数）的梯度
    grads = tape.gradient(loss, variables)
    # TensorFlow自动根据梯度更新参数
    optimizer.apply_gradients(grads_and_vars=zip(grads, variables))

In [75]:
a, b

(<tf.Variable 'a:0' shape=() dtype=float64, numpy=0.9763702100237027>,
 <tf.Variable 'b:0' shape=() dtype=float64, numpy=0.05756498006354141>)